In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "X-Linked_Lymphoproliferative_Syndrome"
cohort = "GSE248835"

# Input paths
in_trait_dir = "../../input/GEO/X-Linked_Lymphoproliferative_Syndrome"
in_cohort_dir = "../../input/GEO/X-Linked_Lymphoproliferative_Syndrome/GSE248835"

# Output paths
out_data_file = "../../output/preprocess/X-Linked_Lymphoproliferative_Syndrome/GSE248835.csv"
out_gene_data_file = "../../output/preprocess/X-Linked_Lymphoproliferative_Syndrome/gene_data/GSE248835.csv"
out_clinical_data_file = "../../output/preprocess/X-Linked_Lymphoproliferative_Syndrome/clinical_data/GSE248835.csv"
json_path = "../../output/preprocess/X-Linked_Lymphoproliferative_Syndrome/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
# 1. Let's first list the directory contents to understand what files are available
import os

print("Files in the cohort directory:")
files = os.listdir(in_cohort_dir)
print(files)

# Adapt file identification to handle different naming patterns
soft_files = [f for f in files if 'soft' in f.lower() or '.soft' in f.lower() or '_soft' in f.lower()]
matrix_files = [f for f in files if 'matrix' in f.lower() or '.matrix' in f.lower() or '_matrix' in f.lower()]

# If no files with these patterns are found, look for alternative file types
if not soft_files:
    soft_files = [f for f in files if f.endswith('.txt') or f.endswith('.gz')]
if not matrix_files:
    matrix_files = [f for f in files if f.endswith('.txt') or f.endswith('.gz')]

print("Identified SOFT files:", soft_files)
print("Identified matrix files:", matrix_files)

# Use the first files found, if any
if len(soft_files) > 0 and len(matrix_files) > 0:
    soft_file = os.path.join(in_cohort_dir, soft_files[0])
    matrix_file = os.path.join(in_cohort_dir, matrix_files[0])
    
    # 2. Read the matrix file to obtain background information and sample characteristics data
    background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
    clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
    background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)
    
    # 3. Obtain the sample characteristics dictionary from the clinical dataframe
    sample_characteristics_dict = get_unique_values_by_row(clinical_data)
    
    # 4. Explicitly print out all the background information and the sample characteristics dictionary
    print("\nBackground Information:")
    print(background_info)
    print("\nSample Characteristics Dictionary:")
    print(sample_characteristics_dict)
else:
    print("No appropriate files found in the directory.")


Files in the cohort directory:
['GSE248835_family.soft.gz', 'GSE248835_series_matrix.txt.gz']
Identified SOFT files: ['GSE248835_family.soft.gz']
Identified matrix files: ['GSE248835_series_matrix.txt.gz']

Background Information:
!Series_title	"Impact of Tumor Microenvironment on Efficacy of CD19 CAR T-Cell Therapy or Chemotherapy and Transplant in Large B-Cell Lymphoma"
!Series_summary	"The phase 3 ZUMA-7 trial in second-line large B-cell lymphoma demonstrated superiority of anti-CD19 CAR T-cell therapy (axicabtagene ciloleucel; axi-cel) over standard of care (SOC; salvage chemotherapy followed by hematopoietic transplantation). Here, we present a prespecified exploratory analysis examining the association between pretreatment tumor characteristics and the efficacy of axi-cel versus SOC. B-cell gene expression signature (GES) and CD19 expression significantly associated with improved event-free survival (EFS) for axi-cel (P=.0002 for B-cell GES; P=.0165 for CD19 expression) but not S

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# 1. Gene Expression Data Availability
# Based on the series title and summary, this appears to be a genomic dataset comparing 
# CD19 CAR T-cell therapy vs standard chemotherapy in lymphoma patients
# The dataset mentions "B-cell gene expression signature" and "CD19 expression"
is_gene_available = True

# 2. Variable Availability and Data Type Conversion

# 2.1 Data Availability
# Examining the sample characteristics dictionary:

# For trait: Looking at treatment arm (index 1)
trait_row = 1  # 'treatment arm' contains info about CAR T-cell therapy vs standard chemotherapy

# For age: Not explicitly provided in sample characteristics
age_row = None  # Age data is not available

# For gender: Not explicitly provided in sample characteristics
gender_row = None  # Gender data is not available

# 2.2 Data Type Conversion Functions
def convert_trait(value_str):
    """Convert treatment arm values to binary trait values.
    0: Standard of Care Chemotherapy (control)
    1: Axicabtagene Ciloleucel (CAR T-cell therapy)
    """
    if value_str is None or pd.isna(value_str):
        return None
    
    # Extract value after colon if present
    if ":" in value_str:
        value = value_str.split(":", 1)[1].strip()
    else:
        value = value_str.strip()
    
    if "Axicabtagene Ciloleucel" in value:
        return 1  # CAR T-cell therapy
    elif "Standard of Care Chemotherapy" in value:
        return 0  # Standard chemotherapy
    else:
        return None

def convert_age(value_str):
    """This function is a placeholder since age data is not available."""
    return None

def convert_gender(value_str):
    """This function is a placeholder since gender data is not available."""
    return None

# 3. Save Metadata
# Determine trait data availability
is_trait_available = trait_row is not None

# Validate and save cohort info (initial filtering)
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
# Note: We need to first load the clinical data from the matrix file
# since there's no prepared clinical_data.csv file
if trait_row is not None:
    # The clinical data will be extracted in a later step after parsing the matrix file
    # For now, we've identified that trait information is available and defined how to convert it
    print(f"Trait information identified at row {trait_row} in sample characteristics")
    print(f"Trait conversion function defined: 1 for Axicabtagene Ciloleucel, 0 for Standard of Care Chemotherapy")
    print(f"Clinical data extraction will be performed after loading the matrix file in subsequent steps")


Trait information identified at row 1 in sample characteristics
Trait conversion function defined: 1 for Axicabtagene Ciloleucel, 0 for Standard of Care Chemotherapy
Clinical data extraction will be performed after loading the matrix file in subsequent steps


### Step 3: Gene Data Extraction

In [4]:
# Use the helper function to get the proper file paths
soft_file_path, matrix_file_path = geo_get_relevant_filepaths(in_cohort_dir)

# Extract gene expression data
try:
    gene_data = get_genetic_data(matrix_file_path)
    
    # Print the first 20 row IDs (gene or probe identifiers)
    print("First 20 gene/probe identifiers:")
    print(gene_data.index[:20])
    
    # Print shape to understand the dataset dimensions
    print(f"\nGene expression data shape: {gene_data.shape}")
    
except Exception as e:
    print(f"Error extracting gene data: {e}")


First 20 gene/probe identifiers:
Index(['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13',
       '14', '15', '16', '17', '18', '19', '20'],
      dtype='object', name='ID')

Gene expression data shape: (817, 256)


### Step 4: Gene Identifier Review

In [5]:
# First, I need to examine whether these identifiers are numeric indices or actual gene symbols
# Based on the preview, these appear to be simple numeric indices (1, 2, 3, etc.)
# These are not human gene symbols, which would typically be alphanumeric like BRCA1, TP53, etc.
# Therefore, these identifiers likely need to be mapped to proper gene symbols

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. This part examines the data more thoroughly to determine what type of data it contains
try:
    # First, let's check a few rows of the gene_data we extracted in Step 3
    print("Sample of gene expression data (first 5 rows, first 5 columns):")
    print(gene_data.iloc[:5, :5])
    
    # Analyze the SOFT file to identify the data type and mapping information
    platform_info = []
    with gzip.open(soft_file_path, 'rt', encoding='latin-1') as f:
        for line in f:
            if line.startswith("!Platform_title") or line.startswith("!Series_title") or "description" in line.lower():
                platform_info.append(line.strip())
    
    print("\nPlatform information:")
    for line in platform_info:
        print(line)
    
    # Extract the gene annotation using the library function
    gene_annotation = get_gene_annotation(soft_file_path)
    
    # Display column names of the annotation dataframe
    print("\nGene annotation columns:")
    print(gene_annotation.columns.tolist())
    
    # Preview the annotation dataframe
    print("\nGene annotation preview:")
    annotation_preview = preview_df(gene_annotation)
    print(annotation_preview)
    
    # Check if ID column exists in the gene_annotation dataframe
    if 'ID' in gene_annotation.columns:
        # Check if any of the IDs in gene_annotation match those in gene_data
        sample_ids = list(gene_data.index[:10])
        matching_rows = gene_annotation[gene_annotation['ID'].isin(sample_ids)]
        print(f"\nMatching rows in annotation for sample IDs: {len(matching_rows)}")
        
        # Look for gene symbol column
        gene_symbol_candidates = [col for col in gene_annotation.columns if 'gene' in col.lower() or 'symbol' in col.lower() or 'name' in col.lower()]
        print(f"\nPotential gene symbol columns: {gene_symbol_candidates}")
    
except Exception as e:
    print(f"Error analyzing gene annotation data: {e}")
    gene_annotation = pd.DataFrame()

# Based on our analysis, determine if this is really gene expression data
# Check the platform description and match with the data we've extracted
is_gene_expression = False
for info in platform_info:
    if 'expression' in info.lower() or 'transcript' in info.lower() or 'mrna' in info.lower():
        is_gene_expression = True
        break

print(f"\nIs this dataset likely to contain gene expression data? {is_gene_expression}")

# If this isn't gene expression data, we need to update our metadata
if not is_gene_expression:
    print("\nNOTE: Based on our analysis, this dataset doesn't appear to contain gene expression data.")
    print("It appears to be a different type of data (possibly SNP array or other genomic data).")
    # Update is_gene_available for metadata
    is_gene_available = False
    
    # Save the updated metadata
    validate_and_save_cohort_info(
        is_final=False,
        cohort=cohort,
        info_path=json_path,
        is_gene_available=is_gene_available,
        is_trait_available=is_trait_available
    )

Sample of gene expression data (first 5 rows, first 5 columns):
    GSM7920866   GSM7920867  GSM7920868  GSM7920869  GSM7920870
ID                                                             
1   428.455036  1478.583496  320.904453  205.785854  131.143273
2    34.272986   136.996953   77.011394   28.266082   58.851203
3     1.830198     1.296121    1.562547    1.741825    2.426707
4    18.455770    19.616558   28.109775   19.053710   21.362373
5     2.899894     1.016105    1.650611    2.173470    0.697614

Platform information:
!Series_title = Impact of Tumor Microenvironment on Efficacy of CD19 CAR T-Cell Therapy or Chemotherapy and Transplant in Large B-Cell Lymphoma
!Platform_title = NanoString Human PanCancer IO360â¢ Panel and Lymphoma Subtyping Test



Gene annotation columns:
['ID', 'Gene_Signature_Name', 'ORF', 'GB_ACC', 'NS Probe ID', 'Analyte Type', 'SPOT_ID']

Gene annotation preview:
{'ID': ['1', '2', '3', '4', '5'], 'Gene_Signature_Name': ['TIS.IO360', 'APM.IO360', 'APM Loss.IO360', 'Apoptosis.IO360', 'ARG1.IO360'], 'ORF': [nan, nan, nan, nan, nan], 'GB_ACC': [nan, nan, nan, nan, nan], 'NS Probe ID': [nan, nan, nan, nan, nan], 'Analyte Type': ['IO360 Signature', 'IO360 Signature', 'IO360 Signature', 'IO360 Signature', 'IO360 Signature'], 'SPOT_ID': ['TIS.IO360', 'APM.IO360', 'APM Loss.IO360', 'Apoptosis.IO360', 'ARG1.IO360']}

Matching rows in annotation for sample IDs: 2570

Potential gene symbol columns: ['Gene_Signature_Name']

Is this dataset likely to contain gene expression data? False

NOTE: Based on our analysis, this dataset doesn't appear to contain gene expression data.
It appears to be a different type of data (possibly SNP array or other genomic data).
